## Temporal Fusion Transformer

In [1]:
import pandas as pd 
import numpy as np
import torch 
from darts import TimeSeries

In [2]:
# Import the data 
def load_and_prepare_data(file_path):
    """
    Load energy prices data from a CSV file, ensure chronological order, and convert 'Date' to datetime.
    """
    df = pd.read_csv(file_path)
    df.sort_values('Date', inplace=True)
    # Convert 'date' column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    #df.set_index('Date', inplace=True)
    df = pd.DataFrame(df)
    return df

In [3]:
# Load in the train and test data
train_df = load_and_prepare_data('../../data/Final_data/train_df.csv')
test_df = load_and_prepare_data('../../data/Final_data/test_df.csv')

# Concatenate the train and test data
df = pd.concat([train_df, test_df])
df['Date'] = pd.to_datetime(df['Date'])

# Create a time series object
series_train = TimeSeries.from_dataframe(train_df, 'Date', 'Day_ahead_price (€/MWh)').astype('float32')
series_test = TimeSeries.from_dataframe(test_df, 'Date', 'Day_ahead_price (€/MWh)').astype('float32')

# Show 
df

,Date,Day_ahead_price (€/MWh),Solar_radiation (W/m2),Wind_speed (m/s),Temperature (°C),Biomass (GWh),Hard_coal (GWh),Hydro (GWh),Lignite (GWh),Natural_gas (GWh),Other (GWh),Pumped_storage_generation (GWh),Solar_energy (GWh),Wind_offshore (GWh),Wind_onshore (GWh),Net_total_export_import (GWh),BEV_vehicles,Oil_price (EUR),TTF_gas_price (€/MWh),Nuclear_energy (GWh)
0,2012-01-01,18.19,14.75,4.95,8.39,98.605,108.454,51.011,325.337,188.811,54.040,19.314,6.263,3.404,235.467,54.662,6,99.64,21.10,250.979
1,2012-01-02,33.82,15.12,5.00,7.41,98.605,222.656,51.862,343.168,229.293,54.166,28.892,6.312,3.350,231.772,-64.477,6,100.04,20.00,258.671
2,2012-01-03,35.03,31.88,7.77,5.23,98.605,162.204,48.851,336.773,241.297,53.518,21.072,24.226,7.292,504.484,-35.078,6,100.44,20.90,271.495
3,2012-01-04,32.16,25.21,8.04,4.78,98.605,189.633,47.101,323.976,252.289,52.194,28.300,14.157,7.828,541.528,22.924,6,103.15,21.40,270.613
4,2012-01-05,20.35,13.46,9.98,4.23,98.605,175.733,45.854,327.502,259.018,52.179,31.887,4.728,8.280,572.819,35.618,6,103.92,21.30,287.555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,2024-07-24,66.61,225.04,3.47,17.54,110.007,43.469,85.857,199.246,194.291,54.026,20.934,325.285,49.360,179.921,-168.705,992,75.75,32.63,0.000
755,2024-07-25,78.34,272.71,2.12,17.85,110.410,50.676,82.632,195.983,209.610,52.963,18.766,394.116,51.053,42.885,-194.496,992,76.36,31.70,0.000
756,2024-07-26,93.04,172.33,2.60,19.09,110.852,42.333,79.531,205.273,205.773,52.616,19.081,256.246,40.449,129.267,-241.786,993,75.21,32.20,0.000
757,2024-07-27,80.74,176.67,2.05,19.63,110.479,33.307,74.958,184.012,216.412,50.927,18.856,244.051,2.180,32.001,-251.655,992,74.79,32.90,0.000


In [4]:
# Define the future covariates columns from your dataframe
future_covariates_columns = ['Solar_radiation (W/m2)', 'Wind_speed (m/s)', 'Temperature (°C)', 
                             'Biomass (GWh)', 'Hard_coal (GWh)', 'Hydro (GWh)', 'Lignite (GWh)', 
                             'Natural_gas (GWh)', 'Other (GWh)', 'Pumped_storage_generation (GWh)', 
                             'Solar_energy (GWh)', 'Wind_offshore (GWh)', 'Wind_onshore (GWh)', 
                             'Net_total_export_import (GWh)', 'BEV_vehicles', 'Oil_price (EUR)', 
                             'TTF_gas_price (€/MWh)', 'Nuclear_energy (GWh)']

## Temporal Fusion Transformer (TFT)

In [5]:
from darts.models import TFTModel
from darts import TimeSeries
from darts.models.forecasting.tft_model import QuantileRegression
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from darts.dataprocessing.transformers import Scaler
from sklearn.preprocessing import MaxAbsScaler, RobustScaler, StandardScaler
from darts.metrics import mape, rmse, mse, mae

# Convert future covariates to TimeSeries objects
future_covariates_train = TimeSeries.from_dataframe(train_df, 'Date', future_covariates_columns).astype('float32')
future_covariates_test = TimeSeries.from_dataframe(test_df, 'Date', future_covariates_columns).astype('float32')

# Convert future covariates to TimeSeries objects
future_covariates_train = TimeSeries.from_dataframe(train_df, 'Date', future_covariates_columns).astype('float32')
future_covariates_full = TimeSeries.from_dataframe(df, 'Date', future_covariates_columns, fill_missing_dates=True, freq="D").astype('float32')

# Determine required start date for future covariates
input_chunk_length = 200  # Set based on desired look-back period
required_start_date = pd.Timestamp(test_df['Date'].iloc[0]) - pd.DateOffset(days=input_chunk_length)

# Ensure future_covariates_full covers the required range
required_end_date = pd.Timestamp(test_df['Date'].iloc[0]) + pd.DateOffset(days=len(series_test)-1)

# Check if future_covariates_full has sufficient data
if future_covariates_full.start_time() > required_start_date or future_covariates_full.end_time() < required_end_date:
    print("Warning: The future_covariates_full is not long enough to cover the required input chunk length and prediction range.")
    # Extend the future_covariates_full or adjust your dataset

# Slice the future covariates to the required range, including data from the training period
future_covariates_test = future_covariates_full.slice(required_start_date, required_end_date)

# Scaling the data
scaler_series = Scaler(MaxAbsScaler())
scaler_covariates = Scaler(MaxAbsScaler())

# Fit the scaler on the training data
series_train_scaled = scaler_series.fit_transform(series_train)
future_covariates_train_scaled = scaler_covariates.fit_transform(future_covariates_train)

# Transform the test series and future covariates using the same scaler
series_test_scaled = scaler_series.transform(series_test)
future_covariates_test_scaled = scaler_covariates.transform(future_covariates_test)

# Define the TFT model
model = TFTModel(
    input_chunk_length=input_chunk_length,  # History length (number of past time steps used to predict the future)
    output_chunk_length=len(series_test),  # Adjusted to predict the length of the test set
    hidden_size=16,
    lstm_layers=4,
    num_attention_heads=4,
    dropout=0.1,
    likelihood=QuantileRegression(quantiles=[0.1, 0.5, 0.9]),  # Correctly using QuantileRegression
    random_state=42,
    add_relative_index=True,  # Automatically generate relative index as a future covariate
    loss_fn=torch.nn.MSELoss()  # Correctly using MSELoss
)

# Train the model with future covariates
model.fit(series_train_scaled, future_covariates=future_covariates_train_scaled, epochs=50, verbose=True)

# Make predictions on the test set
forecast_scaled = model.predict(n=len(series_test), future_covariates=future_covariates_test_scaled)

# Inverse transform the forecast to the original scale
forecast = scaler_series.inverse_transform(forecast_scaled)

# Convert TimeSeries to DataFrame for Plotly plotting
test_df_plotly = series_test.pd_dataframe()
forecast_df_plotly = forecast.pd_dataframe()

# Plot the results using Plotly
fig = go.Figure()

# Add actual test data trace
fig.add_trace(go.Scatter(x=test_df_plotly.index, y=test_df_plotly['Day_ahead_price (€/MWh)'],
                         mode='lines', name='Actual Test Data', line=dict(color='darkblue')))

# Add forecast data trace
fig.add_trace(go.Scatter(x=forecast_df_plotly.index, y=forecast_df_plotly['Day_ahead_price (€/MWh)'],
                         mode='lines', name='TFT Model on Test Data', line=dict(color='red')))

# Update layout
fig.update_layout(
    title='TFT Model - Test Performance Only',
    xaxis_title='Date',
    yaxis_title='Day Ahead Price (€/MWh)',
    legend=dict(
        x=1,   # Set x position to 1 (far right)
        y=1,   # Set y position to 1 (top)
        xanchor='right',  # Anchor the legend's x position to the right
        yanchor='top',    # Anchor the legend's y position to the top
        bordercolor='black',  # Optional: Add a border around the legend
        borderwidth=1        # Optional: Set the border width
    ),
    template='plotly'
)

# Show the plot
fig.show()

# SAve the plot with the number of epochs
fig.write()


# Evaluate the model using Darts' metrics
print(f'Mean Absolute Error on Test Set: {mae(series_test, forecast)}')
print(f'Mean Absolute Percentage Error on Test Set: {mape(series_test, forecast)}')
print(f'Mean Squared Error on Test Set: {mse(series_test, forecast)}')
print(f'Root Mean Squared Error on Test Set: {rmse(series_test, forecast)}')


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

   | Name                              | Type                             | Params | Mode 
------------------------------------------------------------------------------------------------
0  | criterion                         | MSELoss                          | 0      | train
1  | train_criterion                   | MSELoss                          | 0      | train
2  | val_criterion                     | MSELoss                          | 0      | train
3  | train_metrics                     | MetricCollection                 | 0      | train
4  | val_metrics                       | MetricCollection                 | 0      | train
5  | input_embeddings                  | _MultiEmbedding                  | 0      | train
6  | static_covariates_vsn             | _VariableSelectionNetwork        | 0      | train
7  | encoder_vsn                       | _VariableSelection

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

Mean Absolute Error on Test Set: 35.61484146118164
Mean Absolute Percentage Error on Test Set: 61.44070053100586
Mean Squared Error on Test Set: 3861.45849609375
Root Mean Squared Error on Test Set: 62.1406364440918


In [11]:
# Save the created figure as png file and the error metrics 
fig.write_image("../../predictions/Other_Ideas/TFT_epochs_50.png")
error_metrics = pd.DataFrame({'MAE': [mae(series_test, forecast)], 'MAPE': [mape(series_test, forecast)], 'MSE': [mse(series_test, forecast)], 'RMSE': [rmse(series_test, forecast)]})
error_metrics.to_csv('../../predictions/Other_Ideas/TFT_metrics_epochs_50.csv', index=False)


### Hyperparameter Optimiziation 

In [10]:
import optuna
from darts.models import TFTModel
from darts import TimeSeries
from darts.models.forecasting.tft_model import QuantileRegression
import pandas as pd
import numpy as np
import torch
import plotly.graph_objs as go
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mape, rmse, mse, mae

# Convert future covariates to TimeSeries objects
future_covariates_train = TimeSeries.from_dataframe(train_df, 'Date', future_covariates_columns).astype('float32')
future_covariates_test = TimeSeries.from_dataframe(test_df, 'Date', future_covariates_columns).astype('float32')

# Convert full covariates to TimeSeries objects
future_covariates_full = TimeSeries.from_dataframe(df, 'Date', future_covariates_columns, fill_missing_dates=True, freq="D").astype('float32')

# Determine required start date for future covariates
input_chunk_length = 200  # Set based on desired look-back period
required_start_date = pd.Timestamp(test_df['Date'].iloc[0]) - pd.DateOffset(days=input_chunk_length)

# Ensure future_covariates_full covers the required range
required_end_date = pd.Timestamp(test_df['Date'].iloc[0]) + pd.DateOffset(days=len(series_test)-1)

# Check if future_covariates_full has sufficient data
if future_covariates_full.start_time() > required_start_date or future_covariates_full.end_time() < required_end_date:
    print("Warning: The future_covariates_full is not long enough to cover the required input chunk length and prediction range.")
    # Extend the future_covariates_full or adjust your dataset

# Slice the future covariates to the required range, including data from the training period
future_covariates_test = future_covariates_full.slice(required_start_date, required_end_date)

# Scaling the data
scaler_series = Scaler()
scaler_covariates = Scaler()

# Fit the scaler on the training data
series_train_scaled = scaler_series.fit_transform(series_train)
future_covariates_train_scaled = scaler_covariates.fit_transform(future_covariates_train)

# Transform the test series and future covariates using the same scaler
series_test_scaled = scaler_series.transform(series_test)
future_covariates_test_scaled = scaler_covariates.transform(future_covariates_test)

def objective(trial):
    # Suggest hyperparameters
    hidden_size = trial.suggest_int('hidden_size', 8, 64)
    lstm_layers = trial.suggest_int('lstm_layers', 1, 4)
    num_attention_heads = trial.suggest_int('num_attention_heads', 1, 8)
    dropout = trial.suggest_float('dropout', 0.0, 0.5)

    # Initialize the TFT model with suggested hyperparameters
    model = TFTModel(
        input_chunk_length=input_chunk_length,
        output_chunk_length=1, #len(series_test),  # Adjusted to predict the length of the test set
        hidden_size=hidden_size,
        lstm_layers=lstm_layers,
        num_attention_heads=num_attention_heads,
        dropout=dropout,
        likelihood=QuantileRegression(quantiles=[0.1, 0.5, 0.9]),  # Default quantile regression
        random_state=42,
        add_relative_index=True  # Automatically generate relative index as a future covariate
    )

    # Train the model
    model.fit(series_train_scaled, future_covariates=future_covariates_train_scaled, epochs=1, verbose=False)

    # Make predictions
    forecast_scaled = model.predict(n=len(series_test_scaled), future_covariates=future_covariates_test_scaled)

    # Inverse transform the forecast to the original scale
    forecast = scaler_series.inverse_transform(forecast_scaled)

    # Return Mean Squared Error as the objective metric for Optuna to minimize
    return mse(series_test, forecast)


# Run the Optuna study with more trials
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1, n_jobs=1)


# Print the best hyperparameters
print('Best hyperparameters: ', study.best_params)

# Train the final model with best hyperparameters
best_params = study.best_params
best_model = TFTModel(
    input_chunk_length=input_chunk_length,
    output_chunk_length=1, #len(series_test),
    hidden_size=best_params['hidden_size'],
    lstm_layers=best_params['lstm_layers'],
    num_attention_heads=best_params['num_attention_heads'],
    dropout=best_params['dropout'],
    likelihood=QuantileRegression(quantiles=[0.1, 0.5, 0.9]),
    random_state=42,
    add_relative_index=True,
    loss_fn=torch.nn.MSELoss()
)

# Train the best model
best_model.fit(series_train_scaled, future_covariates=future_covariates_train_scaled, epochs=1, verbose=True)

# Make predictions on the test set
forecast_scaled = best_model.predict(n=len(series_test_scaled), future_covariates=future_covariates_test_scaled)

# Inverse transform the forecast to the original scale
forecast = scaler_series.inverse_transform(forecast_scaled)

# Convert TimeSeries to DataFrame for Plotly plotting
test_df_plotly = series_test.pd_dataframe()
forecast_df_plotly = forecast.pd_dataframe()

# Plot the results using Plotly
fig = go.Figure()

# Add actual test data trace
fig.add_trace(go.Scatter(x=test_df_plotly.index, y=test_df_plotly['Day_ahead_price (€/MWh)'],
                         mode='lines', name='Actual Test Data', line=dict(color='darkblue')))

# Add forecast data trace
fig.add_trace(go.Scatter(x=forecast_df_plotly.index, y=forecast_df_plotly['Day_ahead_price (€/MWh)'],
                         mode='lines', name='TFT Forecast on Test Data', line=dict(color='red')))

# Update layout
fig.update_layout(
    title='TFT Model - Test Performance with Optuna Hyperparameter Tuning',
    xaxis_title='Date',
    yaxis_title='Day Ahead Price (€/MWh)',
    legend=dict(
        x=1,   # Set x position to 1 (far right)
        y=1,   # Set y position to 1 (top)
        xanchor='right',  # Anchor the legend's x position to the right
        yanchor='top',    # Anchor the legend's y position to the top
        bordercolor='black',  # Optional: Add a border around the legend
        borderwidth=1        # Optional: Set the border width
    ),
    template='plotly_white'
)

# Show the plot
fig.show()

# Evaluate the model using Darts' metrics
print(f'Mean Absolute Error on Test Set: {mae(series_test, forecast)}')
print(f'Mean Absolute Percentage Error on Test Set: {mape(series_test, forecast)}')
print(f'Mean Squared Error on Test Set: {mse(series_test, forecast)}')
print(f'Root Mean Squared Error on Test Set: {rmse(series_test, forecast)}')


[I 2024-09-11 23:17:23,731] A new study created in memory with name: no-name-733030a1-56b5-4aae-854e-0fa2f6d121b8
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=1` reached.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

[I 2024-09-11 23:19:04,409] Trial 0 finished with value: 15658.7060546875 and parameters: {'hidden_size': 14, 'lstm_layers': 4, 'num_attention_heads': 2, 'dropout': 0.02328583292574732}. Best is trial 0 with value: 15658.7060546875.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

   | Name                              | Type                             | Params | Mode 
------------------------------------------------------------------------------------------------
0  | criterion                         | MSELoss                          | 0      | train
1  | train_criterion                   | MSELoss                          | 0      | train
2  | val_criterion                     | MSELoss                          | 0      | train
3  | train_metrics                     | MetricCollection                 | 0      | train
4  | val_metrics                       | MetricCollection                 | 0      | train
5  | inp

Best hyperparameters:  {'hidden_size': 14, 'lstm_layers': 4, 'num_attention_heads': 2, 'dropout': 0.02328583292574732}


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

Mean Absolute Error on Test Set: 84.37511444091797
Mean Absolute Percentage Error on Test Set: 243.53489685058594
Mean Squared Error on Test Set: 15658.7060546875
Root Mean Squared Error on Test Set: 125.13475036621094
